In [25]:
import pandas as pd
import numpy as np

In [26]:
!pip install scikit-learn==1.2.1

  Using cached scikit-learn-1.2.1.tar.gz (7.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [33 lines of output]
      Traceback (most recent call last):
        File "/Users/gauravjangid/PycharmProjects/insurance project/env001/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 389, in <module>
          main()
        File "/Users/gauravjangid/PycharmProjects/insurance project/env001/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 373, in main
          json_out["return_val"] = hook(**hook_input["kwargs"])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/Users/gauravjangid/PycharmProjects/insurance project/env001/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", l

In [27]:
df=pd.read_csv("/Users/gauravjangid/PycharmProjects/insurance project/Dataset/insurance_processed.csv")


In [28]:
from sklearn.model_selection import train_test_split
x=df.drop(columns=['charges'])
y=df['charges']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)

In [29]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [30]:
df.dtypes

age           int64
sex          object
bmi         float64
children      int64
smoker       object
region       object
charges     float64
dtype: object

In [31]:
preprocessor=ColumnTransformer(transformers=[
('tnf1',OneHotEncoder(drop='first',sparse_output=False,handle_unknown='ignore'),['sex','smoker','region']),
    ('tnf4',StandardScaler(),['age','bmi'])
])

In [32]:
from sklearn.tree import DecisionTreeRegressor
pipeline=Pipeline(steps=[
    ('transformer',preprocessor),
    ('dt_regression',DecisionTreeRegressor())
])


In [33]:
model=pipeline.fit(x_train,y_train)
y_pred=model.predict(x_test)

In [34]:
from sklearn.metrics import r2_score
r2_score(y_pred,y_test)*100

68.21574118210162

In [35]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
ls=[]
for model_train in [LinearRegression,DecisionTreeRegressor,RandomForestRegressor]:
    pipeline=Pipeline(steps=[
    ('transformer',preprocessor),
    ('model',model_train())])
    
    model=pipeline.fit(x_train,y_train)
    y_pred=model.predict(x_test)
    ls.append(model)
    R2_score=r2_score(y_pred,y_test)*100
    print(f'Accuracy {R2_score} {model_train.__name__} ')
    

Accuracy 69.14689625892616 LinearRegression 
Accuracy 68.31972747457485 DecisionTreeRegressor 
Accuracy 82.9651113135006 RandomForestRegressor 


In [36]:
#hyperparameter tuning
from sklearn.model_selection import RandomizedSearchCV

pipeline=ls[2]

param_dist = {
    'model__n_estimators': [50, 100, 200, 300],  # Number of trees
    'model__max_depth': [10, 20, 30, None],  # Depth of trees
    'model__min_samples_split': [2, 5, 10],  # Minimum samples to split
    'model__min_samples_leaf': [1, 2, 4],  # Minimum samples in leaf
}

random_search = RandomizedSearchCV(
    pipeline, param_distributions=param_dist, 
    n_iter=10, cv=5, scoring='r2', n_jobs=-1, random_state=42
)

random_search.fit(x_train, y_train)
best_model = random_search.best_estimator_
y_pred = best_model.predict(x_test)
r2_score(y_pred,y_test)*100


84.6035119472309

In [37]:
import pickle
with open("best_model.pkl","wb") as file:
    pickle.dump(best_model,file)

In [38]:

with open("best_model.pkl","rb") as file:
    pipeline=pickle.load(file)

In [39]:
data=pd.DataFrame([[20,"male",22,0,"yes",'northwest']],columns=['age', 'sex', 'bmi', 'children', 'smoker', 'region'])
pipeline.predict(data)

array([15952.0922139])

In [40]:
data=x_test[:1]
pipeline.predict(data)

array([8812.55073254])

In [41]:
data


,age,sex,bmi,children,smoker,region
764,45,female,25.175,2,no,northeast


In [42]:
import sklearn
print(sklearn.__version__)


1.6.1


In [43]:
df.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges'], dtype='object')

In [44]:
df['region'].unique()

array(['southwest', 'southeast', 'northwest', 'northeast'], dtype=object)